## Show tables with Scala and Spark



In [1]:
spark.catalog.listTables.show(false)

+--------------------------------+--------+-----------+---------+-----------+
|name                            |database|description|tableType|isTemporary|
+--------------------------------+--------+-----------+---------+-----------+
|btl_departures_arrivals_airports|default |null       |EXTERNAL |false      |
|int_airports                    |default |null       |EXTERNAL |false      |
|int_departures                  |default |null       |EXTERNAL |false      |
+--------------------------------+--------+-----------+---------+-----------+



## Use SQL

<div>To display result you need to click on the table symbol after "Out: DataFrame", then execute the new inspection cell...</div><div>For me this is a little bit troublesome in the current Polynote version. I like more using Spark DataFrame's show statement.</div>

In [4]:
select * from default.btl_distances where estarrivalairport = 'LEPA'

[estdepartureairport: string, estarrivalairport: string ... 8 more fields]

In [6]:
spark.sql("select * from default.btl_distances where estarrivalairport = 'LEPA'").show

+-------------------+-----------------+--------------------+----------------+-----------------+-----------------+----------------+-----------------+----------------+---------------------+
|estdepartureairport|estarrivalairport|            arr_name|arr_latitude_deg|arr_longitude_deg|         dep_name|dep_latitude_deg|dep_longitude_deg|        distance|could_be_done_by_rail|
+-------------------+-----------------+--------------------+----------------+-----------------+-----------------+----------------+-----------------+----------------+---------------------+
|               LSZB|             LEPA|Palma De Mallorca...|    39.551700592|    2.73881006241|Bern Belp Airport|    46.914100647|7.497149944309999|904.446224553409|                false|
+-------------------+-----------------+--------------------+----------------+-----------------+-----------------+----------------+-----------------+----------------+---------------------+



In [15]:
spark.table("default.int_departures").where($"estarrivalairport"==="LEPA").show

## Select data by using DataObjects configured in SmartDataLake



In [8]:
// import smartdatalake
import io.smartdatalake.config.SdlConfigObject.stringToDataObjectId
import io.smartdatalake.config.ConfigToolbox
import io.smartdatalake.workflow.dataobject._
import io.smartdatalake.workflow.ActionPipelineContext
import io.smartdatalake.workflow.action.SDLExecutionId
import io.smartdatalake.app.SmartDataLakeBuilderConfig
import io.smartdatalake.workflow.ExecutionPhase
implicit val ss = spark // make Spark session available implicitly

In [9]:
// read config from mounted directory
val (registry, globalConfig) = ConfigToolbox.loadAndParseConfig(Seq("/mnt/config"))
// Create the context used by SDL objects
implicit val context = ActionPipelineContext("test", "app", SDLExecutionId.executionId1, registry, appConfig = SmartDataLakeBuilderConfig("test", Some("app")), phase = ExecutionPhase.Init)

In [10]:
// get a dataobject
val dataIntAirports = registry.get[DeltaLakeTableDataObject]("int-airports")
val dataIntDepartures = registry.get[DeltaLakeTableDataObject]("int-departures")

In [11]:
//dataIntAirports.dropTable

In [12]:
dataIntDepartures.getDataFrame().show

+------+-------------------+--------+-----+
|icao24|estdepartureairport|      dt|count|
+------+-------------------+--------+-----+
|4b4442|               LSZB|20210829|    1|
|4b4449|               LSZB|20210829|    1|
|4d02d7|               LSZB|20210829|    1|
|4b1a01|               LSZB|20210829|    1|
|4b43ab|               LSZB|20210830|    1|
|346603|               LSZB|20210829|    1|
|4b1b13|               LSZB|20210829|    1|
|4b43ab|               LSZB|20210829|    1|
|494108|               LSZB|20210829|    1|
|49d283|               LSZB|20210829|    1|
|4b1b12|               LSZB|20210829|    1|
|4b0f70|               LSZB|20210830|    1|
|4b1f2f|               LSZB|20210830|    1|
|4b4445|               LSZB|20210830|    1|
|aa0da1|               LSZB|20210829|    1|
|4b3049|               LSZB|20210829|    1|
|4b51fa|               LSZB|20210829|    1|
|4b2c71|               LSZB|20210830|    1|
|4b4b8d|               LSZB|20210829|    1|
|44046b|               LSZB|2021